<a href="https://colab.research.google.com/github/miramnair/genai-llama3/blob/main/llama_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nv

In [6]:
pip install git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-wovv82ee
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-wovv82ee
  Resolved https://github.com/huggingface/accelerate.git to commit 4f2c2ba45c4dcdd5c4b441b632529f5017143cc8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [28]:
%%writefile app.py

import streamlit as st
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline
import json
import torch

def getLLAMAresponse(input_text, no_words, blog_style):
    config_data = json.load(open("config.json"))
    huggingface_token = config_data["HUGGINGFACE_TOKEN"]
    #Quantization
    quant_config = BitsAndBytesConfig(
                                      load_in_4bit=True,
                                      bnb_4bit_compute_dtype=torch.bfloat16)

    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B",token=huggingface_token)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B",device_map="auto",quantization_config=quant_config,
                                                token=huggingface_token)
    template = """Write a definition for {blog_style} job profile for a topic {input_text} within {no_words} words."""
    prompt = PromptTemplate(input_variables=["blog_style", "input_text", "no_words"], template = template)
    formatted_prompt = prompt.format(blog_style=blog_style, input_text=input_text, no_words=no_words)

    text_generator = pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=128)
    response = text_generator(formatted_prompt)
    generated_text = response[0]["generated_text"]
    return(generated_text)

# Design the web page

st.set_page_config(page_title="Generate Blogs",
                   layout='centered',
                   initial_sidebar_state='collapsed')

st.header("Generate Article")

input_text = st.text_input("Enter the blog topic")

col1, col2 = st.columns([5, 5])

with col1:
    no_words = st.number_input('No. of words',min_value=1, step=1, value=50,format="%d")

with col2:
    blog_style = st.selectbox('Write article for', ('Researchers', 'Data Scientist'), index=0)

submit = st.button("Generate")

if submit:
    response = getLLAMAresponse(input_text,no_words,blog_style)
    st.write(response)


Overwriting app.py


In [29]:
! wget -q -O - ipv4.icanhazip.com

34.125.38.191


In [30]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.38.191:8501

npx: installed 22 in 1.211s
your url is: https://purple-doodles-count.loca.lt
2024-05-23 15:50:58.291978: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-23 15:50:58.353528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 15:50:58.353574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 15:50:58.355625: E external/local_xla/xla/stream_execut